In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_order_schedule_store
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH d_portal_bu_dc_order_schedule AS (
    SELECT
        a.dc_id,
        b.store_id,
        a.order_schedule
    FROM
        {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_bu_dc_order_schedule a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.business_unit = b.business_unit
    WHERE b.store_status_desc = "Đang hoạt động"
)
SELECT DISTINCT
    IFNULL(a.dc_id, b.dc_id) dc_id,
    IFNULL(a.store_id, b.store_id) store_id,
    IFNULL(a.order_schedule, b.order_schedule) order_schedule
FROM
    {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_dc_store_order_schedule a
    FULL JOIN d_portal_bu_dc_order_schedule b ON a.dc_id = b.dc_id AND a.store_id = b.store_id;
""")